# CNN Visualization in TensorFlow

In [ ]:
import tensorflow as tf

for gpu in tf.config.list_physical_devices('GPU'):
    tf.config.experimental.set_memory_growth(gpu, True)

import numpy as np
import matplotlib.pyplot as plt
import json
import urllib.request

In [ ]:
url = 'https://upload.wikimedia.org/wikipedia/commons/thumb/d/d9/2005_LTI_TXII_Silver_Automatic_2.4_Front.jpg/1920px-2005_LTI_TXII_Silver_Automatic_2.4_Front.jpg'

urllib.request.urlretrieve(url, 'taxi.jpg')

img = tf.keras.preprocessing.image.load_img('taxi.jpg', target_size=(224, 224))

plt.imshow(img);
plt.xticks([]);
plt.yticks([]);

In [ ]:
model = tf.keras.applications.VGG16()
model.summary()

In [ ]:
print('Image:',img.size)
img = tf.keras.preprocessing.image.img_to_array(img)
print('Image:',img.shape)
x = np.expand_dims(img, axis=0)
print('Image:',x.shape)
x = tf.keras.applications.vgg16.preprocess_input(x)
print('Image:',x.shape)

In [ ]:
pred = model.predict(x)
print(pred.shape)

In [ ]:
# plik z klasami ImageNet
url = 'https://raw.githubusercontent.com/pytorch/hub/master/imagenet_classes.txt'
urllib.request.urlretrieve(url, 'imagenet_classes.txt')
    
with open('imagenet_classes.txt', 'r') as f:
    classes = [s.strip() for s in f.readlines()]
    print('Classes:',len(classes))

In [ ]:
max_index = np.argmax(pred[0])
print(max_index)
print(classes[max_index])

In [ ]:
top5 = tf.math.top_k(pred[0],5)
print('\n'.join(['%s %.1f%%'%(classes[i],100*v) for i,v in zip(top5.indices, top5.values)]))

In [ ]:
layers = [layer for layer in model.layers]
names=[layer.name for layer in model.layers]
for i, l in enumerate(layers):
    o=l.output
    print("%2d %25s"%(i, o.name.split(':')[0]), o.shape)

In [ ]:
outputs=[]
image = x
for l in layers:
    if 'flatten' in  l.name:
        break
    image = l(image)
    print(image.shape)
    outputs.append(image)

In [ ]:
layer_id = 1

fig = plt.figure(figsize=(20,20))

plt.suptitle('%s %s\n'%(names[layer_id],str(outputs[layer_id].shape)))

for i in range(outputs[layer_id].shape[-1]):
    _img=outputs[layer_id][0,:,:,i]
    a=fig.add_subplot(8,8,i+1)
    imgplot=plt.imshow(_img,cmap='gray')
    plt.axis('off')
plt.tight_layout()

In [ ]:
layer_id = 18

fig = plt.figure(figsize=(20,10))

plt.suptitle('%s %s\n'%(names[layer_id],str(outputs[layer_id].shape)))

for i in range(outputs[layer_id].shape[-1]):
    _img=outputs[layer_id][0,:,:,i]
    a=fig.add_subplot(16,32,i+1)
    imgplot=plt.imshow(_img,cmap='gray')
    plt.axis('off')
plt.tight_layout()

In [ ]:
first_maps=[]
mean_maps=[]
for feature_map in outputs:
    feature_map=feature_map[0]
    mean_maps.append(tf.math.reduce_mean(feature_map,axis=2))
    first_maps.append(feature_map[:,:,0])
mean_maps[0].shape

In [ ]:
fig =plt.figure(figsize=(20,12))

for i in range(len(first_maps)):
    a=fig.add_subplot(4,8,i+1)
    imgplot=plt.imshow(first_maps[i],cmap='gray')
    plt.axis('off')
    a.set_title('%s\n%dx%d'%(names[i].split('(')[0],first_maps[i].shape[0],first_maps[i].shape[1]), fontsize=20)

In [ ]:
fig =plt.figure(figsize=(20,12))

for i in range(len(mean_maps)):
    a=fig.add_subplot(4,8,i+1)
    imgplot=plt.imshow(mean_maps[i],cmap='gray')
    plt.axis('off')
    a.set_title('%s\n%dx%d'%(names[i].split('(')[0],mean_maps[i].shape[0],mean_maps[i].shape[1]), fontsize=20)